In [50]:
import json
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qfu88\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qfu88\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\qfu88\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:

with open('jobs.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


csv_file_path = 'jobs_full.csv'


if data:
    fieldnames = list(data[0].keys())


with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
   
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
   
    writer.writeheader()
    
  
    for item in data:
        writer.writerow(item)


CSV file created at: jobs_full.csv


In [6]:
df = pd.read_csv('jobs_full.csv')

In [8]:
df.columns

Index(['author', 'author_flair_text', 'clicked', 'created_utc',
       'distinguished', 'edited', 'id', 'is_original_content', 'is_self',
       'link_flair_template_id', 'link_flair_text', 'locked', 'name',
       'num_comments', 'over_18', 'permalink', 'saved', 'score', 'spoiler',
       'stickied', 'text', 'title', 'upvote_ratio', 'url'],
      dtype='object')

In [9]:
df.head(1)

,author,author_flair_text,clicked,created_utc,distinguished,edited,id,is_original_content,is_self,link_flair_template_id,...,over_18,permalink,saved,score,spoiler,stickied,text,title,upvote_ratio,url
0,AutoModerator,NaN,False,1.709510e+09,moderator,False,1b5wc20,False,True,169d3732-5d76-11ea-8708-0e8ff3ab2fbd,...,False,/r/jobs/comments/1b5wc20/success_and_disappoin...,False,8,False,True,This is the weekly success and disappointment...,Success and Disappointment Megathread for the ...,1.0,https://www.reddit.com/r/jobs/comments/1b5wc20...


In [10]:
df.shape

(659, 24)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   author                  659 non-null    object 
 1   author_flair_text       2 non-null      object 
 2   clicked                 659 non-null    bool   
 3   created_utc             659 non-null    float64
 4   distinguished           2 non-null      object 
 5   edited                  659 non-null    object 
 6   id                      659 non-null    object 
 7   is_original_content     659 non-null    bool   
 8   is_self                 659 non-null    bool   
 9   link_flair_template_id  659 non-null    object 
 10  link_flair_text         659 non-null    object 
 11  locked                  659 non-null    bool   
 12  name                    659 non-null    object 
 13  num_comments            659 non-null    int64  
 14  over_18                 659 non-null    bo

In [13]:
df.drop(['author_flair_text', 'distinguished'], axis=1, inplace=True)


In [14]:
df = df.dropna()

df = df.drop_duplicates()

In [18]:
columns_to_drop = ['clicked', 'is_self', 'link_flair_template_id', 'locked', 'saved', 'spoiler', 'stickied']


df = df.drop(columns=columns_to_drop)


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 613 entries, 0 to 658
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   author               613 non-null    object 
 1   created_utc          613 non-null    float64
 2   edited               613 non-null    object 
 3   id                   613 non-null    object 
 4   is_original_content  613 non-null    bool   
 5   link_flair_text      613 non-null    object 
 6   name                 613 non-null    object 
 7   num_comments         613 non-null    int64  
 8   over_18              613 non-null    bool   
 9   permalink            613 non-null    object 
 10  score                613 non-null    int64  
 11  text                 613 non-null    object 
 12  title                613 non-null    object 
 13  upvote_ratio         613 non-null    float64
 14  url                  613 non-null    object 
dtypes: bool(2), float64(2), int64(2), object(9)
m

### Specific Cleaning and Preprocessing



In [26]:
#process created_utc

# Convert 'created_utc' to a datetime column
df['created_datetime'] = pd.to_datetime(df['created_utc'], unit='s')


In [27]:
# "created_utc" feature engineering
df['day_of_week'] = df['created_datetime'].dt.day_name()
df['hour_of_day'] = df['created_datetime'].dt.hour
df['month'] = df['created_datetime'].dt.month_name()
df['year'] = df['created_datetime'].dt.year


In [30]:
## Transformation "edited"

df['was_edited'] = df['edited'].apply(lambda x: False if x == False else True)


In [31]:

df['edit_datetime'] = pd.to_datetime(df['edited'].apply(lambda x: x if x != False else None), unit='s', errors='coerce')


C:\Users\qfu88\AppData\Local\Temp\ipykernel_21944\300110012.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['edit_datetime'] = pd.to_datetime(df['edited'].apply(lambda x: x if x != False else None), unit='s', errors='coerce')


In [33]:
print(df['is_original_content'].unique())
print( df['over_18'].unique())


Unique values in 'is_original_content': [False]
Unique values in 'over_18': [False]


In [34]:
df.drop(['is_original_content', 'over_18'], axis=1, inplace=True)


In [42]:
import re

def basic_text_cleaning(df):
  
    
    url_pattern = r'https?://\S+|www\.\S+'
    html_pattern = r'<.*?>'
    
   
    df = df[~df['text'].isin(['[deleted]', '[removed]'])]
        
    # remove URLs
    df['text'] = df['text'].apply(lambda x: re.sub(url_pattern, '', str(x)))
        
    # remove HTML tags
    df['text'] = df['text'].apply(lambda x: re.sub(html_pattern, '', str(x)))
        
    #whitespace
    df['text'] = df['text'].apply(lambda x: ' '.join(str(x).split()))
        
    # drop rows with very short responses
    df = df[df['text'].apply(lambda x: len(str(x)) > 3)]
        
    return df

# Apply the cleaning function to your DataFrame
df = basic_text_cleaning(df)
df.to_csv('Job_Datasets/df_minimal_cleaning.csv', index=False)

In [46]:

# preprocessing function:
def preprocess_text(text):
   
    # remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # tokenize and lemmatize the text
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stopwords.words('english')]
    # join the tokens back into a string
    return ' '.join(tokens)

In [47]:
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df.to_csv('Job_Datasets/df_nlp_processing.csv', index=False)

In [52]:
df.head(1)

,author,created_utc,edited,id,link_flair_text,name,num_comments,permalink,score,text,title,upvote_ratio,url,created_datetime,day_of_week,hour_of_day,month,year,was_edited,edit_datetime
0,AutoModerator,1.709510e+09,False,1b5wc20,Weekly Megathread,t3_1b5wc20,52,/r/jobs/comments/1b5wc20/success_and_disappoin...,8,this weekly success disappointment Megathread ...,Success and Disappointment Megathread for the ...,1.0,https://www.reddit.com/r/jobs/comments/1b5wc20...,2024-03-04 00:00:17,Monday,0,March,2024,True,NaT


In [53]:
# Drop the original columns that have been processed
df.drop(columns=['created_utc', 'edited'], inplace=True)


In [55]:
tfidf_vect = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000)

# apply it to preprocessed text
tfidf_matrix = tfidf_vect.fit_transform(df['text'])

In [56]:
sparse.save_npz("Job_Datasets/tfidf_matrix.npz", tfidf_matrix)